## Load Library

In [1]:
import math
import time
import itertools

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

## Read File

In [2]:
inFile = open("tsp.txt", "r")

## Convert File to Pandas

In [3]:
arr_list = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    arr_list.append(line.split(" "))
    
df = pd.DataFrame(arr_list, columns = ["Name", "X", "Y"])

# Convert Value From String to int
df["X"] = df["X"].astype(int)
df["Y"] = df["Y"].astype(int)

df

,Name,X,Y
0,Depot,433,204
1,Point1,510,162
2,Parcel2,564,132
3,Parcel3,575,174
4,Point4,422,163
5,Parcel5,345,121
6,Parcel6,392,115
7,Point7,324,226
8,Parcel8,284,217
9,Parcel9,294,268


In [4]:
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

In [5]:
class Point:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"Name: {self.name}, X Pos: {self.x}, Y Pos: {self.y}"

In [6]:
def create_point(ind):
    row = df.iloc[ind, :]
    return Point(row.Name, row.X, row.Y)

In [7]:
def add_edge_to_graph(G, e1, e2, w):
    G.add_edge(e1, e2, weight=w)

## Create Point Array

In [8]:
pt_arr = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    tmp_arr = line.split(" ")
    
    pt_name = tmp_arr[0]
    x_pos = int(tmp_arr[1])
    y_pos = int(tmp_arr[2])
    
    pt = Point(pt_name, x_pos, y_pos)
    
    pt_arr.append(pt)
    
name_arr = list(map(lambda x : x.name, pt_arr))
name_arr

['Depot',
 'Point1',
 'Parcel2',
 'Parcel3',
 'Point4',
 'Parcel5',
 'Parcel6',
 'Point7',
 'Parcel8',
 'Parcel9']

## Create Edge Array

In [9]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    for pt2 in pt_arr:
        dist_val = round(distance(pt, pt2), 3)
        if pt.name == pt2.name or ("Depot" in pt.name and "Point" in pt2.name) or ("Point" in pt.name and "Point" in pt2.name):
            dist_val = float('inf')
        tmp_arr.append(dist_val)
    
    adj_mat.append(tmp_arr)
    
adj_mat

[[inf, inf, 149.482, 145.134, inf, 120.967, 97.99, inf, 149.566, 153.026],
 [87.71, inf, 61.774, 66.098, inf, 170.018, 127.016, inf, 232.596, 240.608],
 [149.482,
  61.774,
  inf,
  43.417,
  145.344,
  219.276,
  172.838,
  257.752,
  292.617,
  302.318],
 [145.134,
  66.098,
  43.417,
  inf,
  153.395,
  236.028,
  192.276,
  256.33,
  294.16,
  296.306],
 [42.45, inf, 145.344, 153.395, inf, 87.71, 56.604, inf, 148.189, 165.557],
 [120.967,
  170.018,
  219.276,
  236.028,
  87.71,
  inf,
  47.381,
  107.079,
  113.741,
  155.596],
 [97.99,
  127.016,
  172.838,
  192.276,
  56.604,
  47.381,
  inf,
  130.173,
  148.553,
  181.695],
 [111.198, inf, 257.752, 256.33, inf, 107.079, 130.173, inf, 41.0, 51.614],
 [149.566,
  232.596,
  292.617,
  294.16,
  148.189,
  113.741,
  148.553,
  41.0,
  inf,
  51.971],
 [153.026,
  240.608,
  302.318,
  296.306,
  165.557,
  155.596,
  181.695,
  51.614,
  51.971,
  inf]]

In [10]:
def gen_adj_mat(adj_mat, column_arr):
    tmp_adj_mat = []
    for row_ind in range(len(adj_mat)):
        if row_ind in column_arr:
            tmp_list = []
            for col_ind in column_arr:
                tmp_list.append(adj_mat[row_ind][col_ind])
            tmp_adj_mat.append(tmp_list)
            
    return tmp_adj_mat

In [11]:
gen_adj_mat(adj_mat, [0, 2, 3, 4, 5])

[[inf, 149.482, 145.134, inf, 120.967],
 [149.482, inf, 43.417, 145.344, 219.276],
 [145.134, 43.417, inf, 153.395, 236.028],
 [42.45, 145.344, 153.395, inf, 87.71],
 [120.967, 219.276, 236.028, 87.71, inf]]

## Create Path

In [12]:
class Path:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost <= other.cost
        
    def __str__(self):
        return f"Shortest Path:{self.path}\nMinimum Cost: {self.cost}"
    
    def __eq__(self, other):
        return self.path == other.path and self.cost == other.cost
    
    def getNodePath(self, name_arr):
        tmp_arr = list(map(lambda x : name_arr[x], self.path))
        return "->".join(tmp_arr)

## Create TSP Matrix

In [13]:
def held_karp(dists, column_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit

            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                    
                # print((bits, k))
                C[(bits, k)] = min(res)

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = ["Depot"] * (n + 1)
    for i in range(n - 1, 0, -1):
        tmp_val = parent
        path[i] = column_arr[tmp_val]
        _, parent = C[(bits, tmp_val)]
        bits = bits & ~(1 << tmp_val)
        
    return opt, path

In [14]:
# held_karp(adj_mat, name_arr)

In [15]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# hk_path = Path([], float('inf'))

# final_start = time.process_time()

# combo_set = set(itertools.combinations(parcel_arr, len(point_arr)))
# total, ind = len(combo_set), 1

# print(f"Total Combinations: {total}")
# for parcel_set in combo_set:
    
#     tmp_list = [0] + [*parcel_set] + point_arr
#     tmp_list = sorted(tmp_list)
    
#     tmp_adj_mat = gen_adj_mat(adj_mat, tmp_list)
    
#     tmp_list = list(map(lambda x : name_arr[x], tmp_list))
    
#     start = time.process_time()
#     val, best_path = held_karp(tmp_adj_mat, tmp_list)
#     print(f"Time Taken (Held-Karp): {time.process_time() - start}")
    
#     hk_path = min(hk_path, Path(best_path, val))
    
#     print(f"Progress: {round(ind / total * 100.0, 2)}%\n")
    
#     ind += 1
    
# print(f"Final Time Taken (Held-Karp): {time.process_time() - final_start}")
# print(hk_path)

## Optimized Held Karp

In [16]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

subset_arr = []

k, l = 1, 1

print(k, l)

subset_dict = {}
for k in range(1, len(point_arr) + 1):
    subset_dict[k] = itertools.combinations(point_arr, k)

1 1


In [17]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

start = time.process_time()

subset_arr = []

for gSubset in itertools.combinations(parcel_arr, 1):
    for rSubset in itertools.combinations(point_arr, 1):
        subset_arr.append(([*gSubset], [*rSubset]))
        
subset_dict = {}
for k in range(1, len(point_arr) + 1):
    subset_dict[k] = [val for val in itertools.combinations(point_arr, k)]
        
for k in range(2, len(point_arr) + 1):
    for gSubset in itertools.combinations(parcel_arr, k):
        for l in range(k - 1, k + 1):
            for rSubset in subset_dict[l]:
                subset_arr.append(([*gSubset], [*rSubset]))
        
subset_arr = sorted(subset_arr, key = lambda x : len(x[0]) + len(x[1]))

print(time.process_time() - start)

0.0


In [18]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

start = time.process_time()

subset_arr_2 = []

for k in range(1, len(point_arr) + 1):
    for gSubset in itertools.combinations(parcel_arr, k):
        for l in range(k - 1, k + 1):
            if l == 0:
                continue
            for rSubset in itertools.combinations(point_arr, l):
                subset_arr.append(([*gSubset], [*rSubset]))
                
subset_arr_2 = sorted(subset_arr_2, key = lambda x : len(x[0]) + len(x[1]))

print(time.process_time() - start)

0.0


In [19]:
print(subset_arr == subset_arr_2)

False


In [20]:
def held_karp(dists, parcel_arr, point_arr, name_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:                             
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}
    
    subset_arr, route_arr = [], []
    for gSubset in parcel_arr:
        # Set transition cost from initial state
        C[(1 << gSubset, gSubset)] = (dists[0][gSubset], 0)
        
        for rSubset in point_arr:
            subset_arr.append(([gSubset], [rSubset]))
        
    subset_dict = {}
    for k in range(1, len(point_arr) + 1):
        subset_dict[k] = [val for val in itertools.combinations(point_arr, k)]
        
    for k in range(2, len(point_arr) + 1):
        for gSubset in itertools.combinations(parcel_arr, k):
            for l in range(k - 1, k + 1):
                for rSubset in subset_dict[l]:
                    subset_arr.append(([*gSubset], [*rSubset]))
                    
                    if l + k == 2 * len(point_arr):
                        bits = 0
                        for bit in gSubset:
                            bits |= 1 << bit
            
                        for bit in rSubset:
                            bits |= 1 << bit
                        
                        route_arr.append(bits)
                
#     subset_arr = sorted(subset_arr, key = lambda x : len(x[0]) + len(x[1]))

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset in subset_arr:
        gSubset, rSubset = subset
        # Set bits for all nodes in this subset
        bits = 0
        
        for bit in gSubset:
            bits |= 1 << bit
            
        for bit in rSubset:
            bits |= 1 << bit
        
        subset2 = []
        
        if len(gSubset) == len(rSubset):
            # The last node FROM THE PREVIOUS SUBSET will be a red point
            subset2 = gSubset
        else:
            # The last node FROM THE PREVIOUS SUBSET will be a green point
            subset2 = rSubset
            
        subset1 = [0] * (len(gSubset) + len(rSubset))
        
        ind = 0
        for elem in gSubset:
            subset1[ind] = elem
            ind += 1
            
        for elem in rSubset:
            subset1[ind] = elem
            ind += 1

        # Find the lowest cost to get to this subset
        for k in subset1:
            prev = bits & ~(1 << k)

            opt, parent = float('inf'), -1
            flag = False
            
            for m in subset2:
                # Check if Key Exist
                key = (prev, m)
                if key in C:
                    flag = True
                    tmp_opt = C[key][0] + dists[m][k]
                    if tmp_opt <= opt:
                        opt = tmp_opt
                        parent = m
                  
            if flag:
                C[(bits, k)] = (opt, parent)
            
    bits = 0
    opt = float('inf')
    parent = 0

    # Calculate optimal cost
    for route_bits in route_arr:
        for k in point_arr:
            tmp_opt = C[(route_bits, k)][0] + dists[k][0]
            
            if tmp_opt <= opt:
                opt = tmp_opt
                parent = k
                bits = route_bits

    # Backtrack to find full path
    path = ["Depot"] * (2 * len(point_arr) + 2)
    for i in range(2 * len(point_arr), 0, -1):
        path[i] = name_arr[parent]
        key = (bits, parent)
        bits = bits & ~(1 << parent)
        _, parent = C[key]
        
    return opt, path

In [21]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
print(parcel_arr, point_arr)

[2, 3, 5, 6, 8, 9] [1, 4, 7]


In [22]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
start = time.process_time()
val, best_path = held_karp(adj_mat, parcel_arr, point_arr, name_arr)
hk2_path = Path(best_path, val)
print(hk2_path)
print(time.process_time() - start)

Shortest Path:['Depot', 'Parcel8', 'Point7', 'Parcel5', 'Point4', 'Parcel6', 'Point1', 'Depot']
Minimum Cost: 656.685
0.0


In [23]:
# print(hk_path == hk2_path)